In [24]:
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import re
from matplotlib.widgets import Button
matplotlib.use('Qt5Agg')

def parse_input_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    grid_line = lines[0]
    grid_width, grid_height, *_ = map(int, grid_line.split(','))
    obs_lines, pin_lines = [], []
    for line in lines[1:]:
        if line.startswith("OBS"):
            obs_lines.append(line)
        else:
            pin_lines.append(line)
    obstacles = [tuple(map(int, re.search(r'\((\d+),\s*(\d+),\s*(\d+)\)', l).groups())) for l in obs_lines]

    pins, net_names = [], []
    for line in pin_lines:
        tokens = line.split()
        net_names.append(tokens[0])
        coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
        pins.append([tuple(map(int, c)) for c in coords])
    return (grid_width, grid_height), obstacles, pins, net_names

def parse_output_file(filename):
    output_nets = []
    with open(filename, 'r') as f:
        for line in f:
            coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
            output_nets.append([tuple(map(int, c)) for c in coords])
    return output_nets

def visualize(grid_size, obstacles, input_nets, output_nets, net_names=None):
    grid_width, grid_height = grid_size
    all_layers = sorted(set(p[0] for net in output_nets for p in net))
    cmap = plt.colormaps.get_cmap('tab20')

    #fig, axs = plt.subplots(1, len(all_layers), figsize=(12 * len(all_layers), 12), dpi=150)
    fig, axs = plt.subplots(1, len(all_layers), figsize=(8 * len(all_layers), 8), layout="constrained")

    if len(all_layers) == 1:
        axs = [axs]
    layer_to_ax = dict(zip(all_layers, axs))

    layer_artists = {layer: [] for layer in all_layers}

    def draw_grid(ax):
        ax.grid(False)
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        x_range = xlim[1] - xlim[0]
        y_range = ylim[1] - ylim[0]
        step = max(1, int(min(x_range, y_range) // 20))
        ax.set_xticks(range(int(xlim[0]), int(xlim[1]) + 1, step))
        ax.set_yticks(range(int(ylim[0]), int(ylim[1]) + 1, step))
        ax.grid(True, linestyle=':', linewidth=0.8)

    for layer, ax in layer_to_ax.items():
        ax.set_title(f"Layer {layer}")
        ax.set_aspect('equal')
        ax.set_facecolor('#f0f0f0')
        ax.add_patch(patches.Rectangle(
            (0, 0), grid_width, grid_height,
            edgecolor='black', facecolor='white', linewidth=2, zorder=0
        ))
        ax.set_xlim(-1, grid_width + 1)
        ax.set_ylim(-1, grid_height + 1)
        draw_grid(ax)

    for (layer, ox, oy) in obstacles:
        ax = layer_to_ax.get(layer)
        if ax:
            r = patches.Rectangle((ox, oy), 1, 1, color='black', zorder=1)
            ax.add_patch(r)
            layer_artists[layer].append(r)

    for net_idx, net in enumerate(output_nets):
        color = cmap(net_idx)
        net_label = net_names[net_idx] if net_names else f"net{net_idx+1}"
        layers_labeled = set()
        for (layer, x, y) in net:
            ax = layer_to_ax[layer]
            label = net_label if (net_label, layer) not in layers_labeled else None
            r = patches.Rectangle((x, y), 1, 1, color=color, zorder=1, label=label)
            ax.add_patch(r)
            layer_artists[layer].append(r)
            layers_labeled.add((net_label, layer))

    used_via_layers = set()
    for net_idx, net in enumerate(output_nets):
        net_color = cmap(net_idx)
        for i in range(len(net) - 1):
            l1, x1, y1 = net[i]
            l2, x2, y2 = net[i + 1]
            if l1 != l2:
                for l in [l1, l2]:
                    ax = layer_to_ax.get(l)
                    if ax:
                        label = "Via" if l not in used_via_layers else None
                        r = patches.Rectangle((x2 + 0.27, y2 + 0.27), 0.45, 0.45,
                                              linewidth=1.2, edgecolor='black',
                                              facecolor='red', zorder=6, label=label)
                        ax.add_patch(r)
                        layer_artists[l].append(r)
                        if label:
                            used_via_layers.add(l)

    used_light_pin_labels_per_layer = set()
    used_other_pin_labels_per_layer = set()
    for net_idx, net in enumerate(input_nets):
        for i, (layer, x, y) in enumerate(net):
            ax = layer_to_ax.get(layer)
            if ax:
                if i == 0:
                    label_key = (layer, "Input Pin")
                    label = "Input Pin" if label_key not in used_light_pin_labels_per_layer else None
                    r = patches.Rectangle((x + 0.15, y + 0.15), 0.7, 0.7,
                                          linewidth=1, edgecolor='black',
                                          facecolor='#90ee90', zorder=4, label=label)
                    ax.add_patch(r)
                    layer_artists[layer].append(r)
                    used_light_pin_labels_per_layer.add(label_key)
                else:
                    label_key = (layer, "Output Pin")
                    label = "Output Pin" if label_key not in used_other_pin_labels_per_layer else None
                    r = patches.Rectangle((x + 0.15, y + 0.15), 0.7, 0.7,
                                          linewidth=1, edgecolor='black',
                                          facecolor='green', zorder=4, label=label)
                    ax.add_patch(r)
                    layer_artists[layer].append(r)
                    used_other_pin_labels_per_layer.add(label_key)

    def set_initial_limits():
        for ax_ in axs:
            ax_.set_xlim(-1, grid_width + 1)
            ax_.set_ylim(-1, grid_height + 1)
            draw_grid(ax_)
            ax_.figure.canvas.draw_idle()

    def draw_grid(ax):
        ax.grid(False)
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        x_range = xlim[1] - xlim[0]
        y_range = ylim[1] - ylim[0]
        step = max(1, int(min(x_range, y_range) // 20))
        ax.set_xticks(range(int(xlim[0]), int(xlim[1]) + 1, step))
        ax.set_yticks(range(int(ylim[0]), int(ylim[1]) + 1, step))
        ax.grid(True, linestyle=':', linewidth=0.8)

    for ax in axs:
        set_initial_limits()
        draw_grid(ax)

    def on_scroll(event):
        base_scale = 1.2
        if event.xdata is None or event.ydata is None:
            return
        scale_factor = 1 / base_scale if event.button == 'up' else base_scale
        for ax in axs:
            cur_xlim = ax.get_xlim()
            cur_ylim = ax.get_ylim()
            xdata = event.xdata
            ydata = event.ydata
            new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
            new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor
            relx = (xdata - cur_xlim[0]) / (cur_xlim[1] - cur_xlim[0])
            rely = (ydata - cur_ylim[0]) / (cur_ylim[1] - cur_ylim[0])
            ax.set_xlim([xdata - new_width * relx, xdata + new_width * (1 - relx)])
            ax.set_ylim([ydata - new_height * rely, ydata + new_height * (1 - rely)])
            draw_grid(ax)
        fig.canvas.draw_idle()

    def on_press(event):
        for ax in axs:
            ax._pan_start = (event.x, event.y, ax.get_xlim(), ax.get_ylim())

    def on_motion(event):
        if event.x is None or event.y is None:
            return
        for ax in axs:
            if not hasattr(ax, '_pan_start') or ax._pan_start is None:
                continue
            x_press, y_press, xlim, ylim = ax._pan_start
            dx = event.x - x_press
            dy = event.y - y_press
            scale_x = (xlim[1] - xlim[0]) / ax.bbox.width
            scale_y = (ylim[1] - ylim[0]) / ax.bbox.height
            ax.set_xlim(xlim[0] - dx * scale_x, xlim[1] - dx * scale_x)
            ax.set_ylim(ylim[0] - dy * scale_y, ylim[1] - dy * scale_y)
            draw_grid(ax)
        fig.canvas.draw_idle()
    def on_reset(event):
        set_initial_limits()
        

    def on_release(event):
        for ax in axs:
            ax._pan_start = None
    reset_ax = plt.axes([0.45, 0.01, 0.1, 0.04])  # x, y, width, height in figure coords
    reset_button = Button(reset_ax, 'Reset View', hovercolor='0.975')
    # Connect only zoom and pan
    reset_button.on_clicked(on_reset)
    fig.canvas.mpl_connect('scroll_event', on_scroll)
    fig.canvas.mpl_connect('button_press_event', on_press)
    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    fig.canvas.mpl_connect('button_release_event', on_release)
    
    # Legend
    handles, labels = axs[0].get_legend_handles_labels()
    legend_handles = {}
    for h, l in zip(handles, labels):
        if l not in legend_handles:
            legend_handles[l] = h
    #fig.legend(legend_handles.values(), legend_handles.keys(), loc='center right', fontsize='small', borderaxespad=0.1)
    fig.legend(legend_handles.values(), legend_handles.keys(), loc='upper right', fontsize='small', bbox_to_anchor=(1.0, 1.0),borderaxespad=0.1)


    fig.tight_layout(rect=[0, 0.1, 1, 0.95])
    plt.show()

            

if __name__ == "__main__":
    input_file = r"tests\input_1000x1000_1.txt"
    output_file = r"tests\output_1000x1000_1.txt"
    # input_file = r"tests_2\input_5x5_2.txt"
    # output_file = r"tests_2\output_5x5_2.txt"
    grid_size, obstacles, input_nets, net_names = parse_input_file(input_file)
    output_nets = parse_output_file(output_file)
    visualize(grid_size, obstacles, input_nets, output_nets, net_names)

C:\Users\PC\AppData\Local\Temp\ipykernel_23664\1427734385.py:223: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.1, 1, 0.95])
C:\Users\PC\AppData\Local\Temp\ipykernel_23664\1427734385.py:223: UserWarning: The figure layout has changed to tight
  fig.tight_layout(rect=[0, 0.1, 1, 0.95])


In [32]:
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import re
import numpy as np
from matplotlib.widgets import Button
from matplotlib.collections import PatchCollection,LineCollection
matplotlib.use('Qt5Agg')

def parse_input_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    grid_line = lines[0]
    grid_width, grid_height, *_ = map(int, grid_line.split(','))
    obs_lines, pin_lines = [], []
    for line in lines[1:]:
        if line.startswith("OBS"):
            obs_lines.append(line)
        else:
            pin_lines.append(line)
    obstacles = [tuple(map(int, re.search(r'\((\d+),\s*(\d+),\s*(\d+)\)', l).groups())) for l in obs_lines]

    pins, net_names = [], []
    for line in pin_lines:
        tokens = line.split()
        net_names.append(tokens[0])
        coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
        pins.append([tuple(map(int, c)) for c in coords])
    return (grid_width, grid_height), obstacles, pins, net_names

def parse_output_file(filename):
    output_nets = []
    with open(filename, 'r') as f:
        for line in f:
            coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
            output_nets.append([tuple(map(int, c)) for c in coords])
    return output_nets


def visualize(grid_size, obstacles, input_nets, output_nets, net_names=None):
    grid_width, grid_height = grid_size
    all_layers = sorted(set(p[0] for net in output_nets for p in net))
    cmap = plt.get_cmap('tab20')
    pan_state = {'press': None}


    fig, axs = plt.subplots(1, len(all_layers), figsize=(8 * len(all_layers), 8), layout='constrained')
    if len(all_layers) == 1:
        axs = [axs]
    layer_to_ax = dict(zip(all_layers, axs))

    def draw_grid(ax, grid_width, grid_height):
        # Fast grid lines
        step = 1
        x_lines = [[(x, 0), (x, grid_height)] for x in range(0, grid_width + 1, step)]
        y_lines = [[(0, y), (grid_width, y)] for y in range(0, grid_height + 1, step)]
        lines = LineCollection(x_lines + y_lines, colors='lightgray', linewidths=0.3, zorder=0)
        ax.add_collection(lines)

    for layer, ax in layer_to_ax.items():
        ax.set_title(f"Layer {layer}")
        ax.set_xlim(-1, grid_width + 1)
        ax.set_ylim(-1, grid_height + 1)
        ax.set_aspect('equal')
        ax.set_facecolor('#f0f0f0')
        draw_grid(ax, grid_width, grid_height)

    # Draw obstacles using scatter
    for layer in all_layers:
        layer_obs = [(x, y) for (l, x, y) in obstacles if l == layer]
        ax = layer_to_ax[layer]
        for (x, y) in layer_obs:
            rect = Rectangle((x, y), 1, 1, facecolor='black', edgecolor=None, zorder=1)
            ax.add_patch(rect)


    legend_handles = []

    for net_idx, net in enumerate(output_nets):
        color = cmap(net_idx)
        net_label = net_names[net_idx] if net_names and net_idx < len(net_names) else f"Net {net_idx}"
        added_to_legend = False

        for (layer, x, y) in net:
            ax = layer_to_ax[layer]
            rect = Rectangle((x, y), 1, 1, facecolor=color, edgecolor='k', linewidth=0.5, alpha=0.6, zorder=2)
            ax.add_patch(rect)

            if not added_to_legend:
                proxy = Rectangle((0, 0), 1, 1, facecolor=color, edgecolor='k', alpha=0.6)
                legend_handles.append((proxy, net_label))
                added_to_legend = True

    
    base_marker_size = 500 
    scale_factor = max(grid_width, grid_height)
    scale_factor = min(scale_factor, 10)

    via_size = base_marker_size / (scale_factor)
    pin_size = base_marker_size / (scale_factor-7)

    # Draw vias
    via_points_per_layer = {layer: [] for layer in all_layers}
    for net in output_nets:
        for i in range(len(net) - 1):
            l1, x1, y1 = net[i]
            l2, x2, y2 = net[i + 1]
            if l1 != l2:
                for l in [l1, l2]:
                    via_points_per_layer[l].append((x2 + 0.5, y2 + 0.5))
    for layer, points in via_points_per_layer.items():
        if points:
            px, py = zip(*points)
            layer_to_ax[layer].scatter(px, py, s=via_size, c='red', edgecolors='black', linewidths=0.8, marker='o', zorder=5, label='Via')

    # Draw pins
    input_pin_locs = {layer: [] for layer in all_layers}
    output_pin_locs = {layer: [] for layer in all_layers}
    for net in input_nets:
        for i, (layer, x, y) in enumerate(net):
            pin_dict = input_pin_locs if i == 0 else output_pin_locs
            pin_dict[layer].append((x + 0.5, y + 0.5))

    for layer, points in input_pin_locs.items():
        if points:
            px, py = zip(*points)
            layer_to_ax[layer].scatter(px, py, s=pin_size, c='#90ee90', edgecolors='black', linewidths=1, zorder=4, label='Input Pin')
    for layer, points in output_pin_locs.items():
        if points:
            px, py = zip(*points)
            layer_to_ax[layer].scatter(px, py, s=pin_size, c='green', edgecolors='black', linewidths=1, zorder=4, label='Output Pin')

    def on_scroll(event):
            base_scale = 1.2
            if event.xdata is None or event.ydata is None:
                return
            scale_factor = 1 / base_scale if event.button == 'up' else base_scale
            x, y = event.xdata, event.ydata

            for ax in axs:
                xlim = ax.get_xlim()
                ylim = ax.get_ylim()
                dx = x - xlim[0]
                dy = y - ylim[0]
                new_xlim = [x - dx * scale_factor, x + (xlim[1] - x) * scale_factor]
                new_ylim = [y - dy * scale_factor, y + (ylim[1] - y) * scale_factor]
                ax.set_xlim(new_xlim)
                ax.set_ylim(new_ylim)

            fig.canvas.draw_idle()

    def on_motion(event):
        if pan_state['press'] is None or event.x is None or event.y is None:
            return
        dx = event.x - pan_state['press'][0]
        dy = event.y - pan_state['press'][1]

        for ax in axs:
            xlim = ax.get_xlim()
            ylim = ax.get_ylim()
            scale_x = (xlim[1] - xlim[0]) / ax.bbox.width
            scale_y = (ylim[1] - ylim[0]) / ax.bbox.height
            ax.set_xlim(xlim[0] - dx * scale_x, xlim[1] - dx * scale_x)
            ax.set_ylim(ylim[0] - dy * scale_y, ylim[1] - dy * scale_y)

        pan_state['press'] = (event.x, event.y)
        fig.canvas.draw_idle()

    def on_reset(event=None):
        for ax in axs:
            ax.set_xlim(-1, grid_width + 1)
            ax.set_ylim(-1, grid_height + 1)

            # Re-draw the grid
            draw_grid(ax, grid_width, grid_height)

        fig.canvas.draw_idle()

    def on_press(event):
        if event.button == 1:
            pan_state['press'] = (event.x, event.y)

    def on_release(event):
        if event.button == 1:
            pan_state['press'] = None

    fig.canvas.mpl_connect('scroll_event', on_scroll)
    fig.canvas.mpl_connect('button_press_event', on_press)
    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    fig.canvas.mpl_connect('button_release_event', on_release)

    # Reset View Button
    reset_ax = plt.axes([0.45, 0.01, 0.1, 0.04])
    reset_button = Button(reset_ax, 'Reset View', hovercolor='0.975')
    reset_button.on_clicked(on_reset)

    # Collect scatter legend entries from axs[0]
    handles, labels = axs[0].get_legend_handles_labels()

    # Add proxy handles from nets (if any)
    if legend_handles:
        proxy_handles, proxy_labels = zip(*legend_handles)
        handles += proxy_handles
        labels += proxy_labels

    # Deduplicate
    seen = {}
    for h, l in zip(handles, labels):
        if l not in seen:
            seen[l] = h

    # Final legend
    fig.legend(seen.values(), seen.keys(), loc='upper right', fontsize='small',
            bbox_to_anchor=(1.0, 1.0), borderaxespad=0.1)


    fig.tight_layout(rect=[0, 0.1, 1, 0.95])
    plt.show()

            

if __name__ == "__main__":
    input_file = r"tests\input_1000x1000_1.txt"
    output_file = r"tests\output_1000x1000_1.txt"
    # input_file = r"tests_2\input_5x5_2.txt"
    # output_file = r"tests_2\output_5x5_2.txt"
    grid_size, obstacles, input_nets, net_names = parse_input_file(input_file)
    output_nets = parse_output_file(output_file)
    visualize(grid_size, obstacles, input_nets, output_nets, net_names)

C:\Users\PC\AppData\Local\Temp\ipykernel_23664\3322355893.py:218: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.1, 1, 0.95])
C:\Users\PC\AppData\Local\Temp\ipykernel_23664\3322355893.py:218: UserWarning: The figure layout has changed to tight
  fig.tight_layout(rect=[0, 0.1, 1, 0.95])
